In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/A_New_Perspective_for_Shuttlecock_Hitting_Event_Detection/'

Mounted at /content/gdrive
/content/gdrive/MyDrive/A_New_Perspective_for_Shuttlecock_Hitting_Event_Detection


In [2]:
import os
import cv2
import numpy as np
import pandas as pd

os.mkdir('./preprocess')
os.mkdir('./preprocess/Backhand')
os.mkdir('./preprocess/Backhand/0')
os.mkdir('./preprocess/Backhand/1')

trainPath = './data/part1/train/'
trainFolders = os.listdir(trainPath)
trainFolders = sorted(trainFolders)

savePath = './preprocess/Backhand/'
saveFolder_1 = savePath + '0/'
saveFolder_2 = savePath +'1/'


for folder in trainFolders:
    fileName = os.listdir(trainPath + folder + '/')
    for fn in fileName:
        if fn[-1] == '4':
            videoPath = trainPath + folder + '/' + fn
        if fn[-1] == 'v':
            data = pd.read_csv(trainPath + folder + '/' + fn)
            hits = [a for a in data['HitFrame']]
            hitter = [b for b in data['Backhand']]

    cap = cv2.VideoCapture(videoPath)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    i = 0
    for i in range(frame_count):
        ret, frame = cap.read()

        frame = frame[:, 280:1280-280]

        if ret:
            if i in set(hits):
                if hitter[hits.index(i)] == 1:
                    cv2.imwrite(saveFolder_1 + folder + '_' + str(i) + '.jpg', frame)
                else:
                    cv2.imwrite(saveFolder_2 + folder + '_' + str(i) + '.jpg', frame)
        i+=1

    cap.release()

# check
print(len(os.listdir(saveFolder_1)))
print(len(os.listdir(saveFolder_2)))

19
41


In [3]:
import os
import csv
import shutil
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

HitterPath = './preprocess/Backhand/'
HitterFolder = os.listdir(HitterPath)
HitterFolder = sorted(HitterFolder)

Hitter_A_Path = './preprocess/Backhand/0/'
Hitter_A_Files = os.listdir(Hitter_A_Path)
Hitter_A_Files = sorted(Hitter_A_Files)

Hitter_B_Path = './preprocess/Backhand/1/'
Hitter_B_Files = os.listdir(Hitter_B_Path)
Hitter_B_Files = sorted(Hitter_B_Files)

filename = Hitter_A_Files + Hitter_B_Files
category = ['0' for i in range(len(Hitter_A_Files))] + ['1' for i in range(len(Hitter_B_Files))]
filename = np.array(filename, dtype=object)
category = np.array(category, dtype=object)

X_rem, y_rem = filename, category

Names = ['train', 'val']

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=113)

n = 1
for train_index, test_index in skf.split(X_rem, y_rem):
    X_train, X_val = X_rem[train_index], X_rem[test_index]
    y_train, y_val = y_rem[train_index], y_rem[test_index]

    Data_x = [X_train, X_val]
    Data_y = [y_train, y_val]


    savePath = './preprocess/Backhand/'
    foldPath = os.path.join(savePath, 'fold' + str(n))
    os.mkdir(foldPath)

    # write csv files
    for i in range(len(Names)):
        with open(foldPath + '/' + Names[i] + '.csv', 'w') as f:
            writer = csv.writer(f)
            zipped_lists = zip(Data_x[i], Data_y[i])
            for row in zipped_lists:
                writer.writerow(row)

        path1 = os.path.join(foldPath + '/', Names[i])
        os.mkdir(path1)


        for it, cl in zip(Data_x[i], Data_y[i]):
            imgName = HitterPath + cl + '/' + it
            path2 = os.path.join(path1, cl)
            try:
                os.mkdir(path2)
            except:
                pass
            shutil.copy(imgName, path2 + '/' + it)
    n+=1

In [4]:
!git clone https://github.com/NVIDIA/apex
%cd './apex/'
!python3 setup.py install
%cd '/content/gdrive/MyDrive/A_New_Perspective_for_Shuttlecock_Hitting_Event_Detection/'
!git clone https://github.com/jeonsworld/ViT-pytorch.git
%cd './ViT-pytorch/'
!mv '/content/gdrive/MyDrive/A_New_Perspective_for_Shuttlecock_Hitting_Event_Detection/src/requirements.txt' '/content/gdrive/MyDrive/A_New_Perspective_for_Shuttlecock_Hitting_Event_Detection/ViT-pytorch/requirements.txt'
!mv '/content/gdrive/MyDrive/A_New_Perspective_for_Shuttlecock_Hitting_Event_Detection/src/train.py' '/content/gdrive/MyDrive/A_New_Perspective_for_Shuttlecock_Hitting_Event_Detection/ViT-pytorch/train.py'
!mv '/content/gdrive/MyDrive/A_New_Perspective_for_Shuttlecock_Hitting_Event_Detection/src/data_utils.py' '/content/gdrive/MyDrive/A_New_Perspective_for_Shuttlecock_Hitting_Event_Detection/ViT-pytorch/utils/data_utils.py'
!pip install -q -r requirements.txt
os.mkdir('./checkpoint')
%cd './checkpoint/'
!wget https://storage.googleapis.com/vit_models/imagenet21k+imagenet2012/ViT-B_16.npz

Cloning into 'apex'...
remote: Enumerating objects: 11632, done.
remote: Counting objects: 100% (3700/3700), done.
remote: Compressing objects: 100% (563/563), done.
remote: Total 11632 (delta 3341), reused 3264 (delta 3134), pack-reused 7932
Receiving objects: 100% (11632/11632), 15.46 MiB | 13.26 MiB/s, done.
Resolving deltas: 100% (8170/8170), done.
Updating files: 100% (495/495), done.
/content/gdrive/MyDrive/A_New_Perspective_for_Shuttlecock_Hitting_Event_Detection/apex


torch.__version__  = 2.1.0+cu121


running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for deta

In [5]:
%cd '/content/gdrive/MyDrive/A_New_Perspective_for_Shuttlecock_Hitting_Event_Detection/ViT-pytorch/'
!python train.py --name Backhand --dataset Backhand --foldn 1 --model_type ViT-B_16 --pretrained_dir checkpoint/ViT-B_16.npz --train_batch_size 4 --img_size 480 --fp16 --fp16_opt_level O2 --num_steps 1000

/content/gdrive/MyDrive/A_New_Perspective_for_Shuttlecock_Hitting_Event_Detection/ViT-pytorch
2024-02-14 03:43:30.366092: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-14 03:43:30.366156: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-14 03:43:30.368135: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-14 03:43:30.384741: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorF